Principle Componenet Analysis
Finding the directions of maximum variance in high-dimensional data and project it onto a smaller dimensional subspace while retaining most of the information.

### A Summary of the PCA Approach

 - Standardize the data.
 
 - Obtain the Eigenvectors and Eigenvalues from the covariance matrix or correlation matrix, or    perform Singular Vector Decomposition.
 
 - Sort eigenvalues in descending order and choose the k eigenvectors that correspond to the k         largest eigenvalues where k is the number of dimensions of the new feature  subspace (k≤d).
 
 - Construct the projection matrix W from the selected k eigenvectors.
 
 - Transform the original dataset X via W to obtain a k-dimensional feature subspace Y

    .




In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


In [ ]:
iris = load_iris()
X = iris.data
y = iris.target
X.shape
#print(X)
#print(y)

In [ ]:
X_std = StandardScaler().fit_transform(X)
#print(X_std)
print(X_std.shape)


### 1 - Eigendecomposition - Computing Eigenvectors and Eigenvalues

The eigenvectors (principal components) determine the directions of the new feature space, and the eigenvalues determine their magnitude. 

In other words, the eigenvalues explain the variance of the data along the new feature axes.

In [ ]:
#Covariance Matrix
import numpy as np
mean_vec = np.mean(X_std,axis=0)
#print(mean_vec)
cov_mat = (X_std - mean_vec).T.dot(X_std - mean_vec)/(X_std.shape[0]-1)
print ('Covariance matrix')
print (cov_mat)

In [ ]:
#Alternative
cov_mat=np.cov(X_std.T)
cov_mat

### find eigenvalues and eigenvectors

In [ ]:
eig_vals,eig_vecs = np.linalg.eig(cov_mat)

print('Eigenvectors \n%s' %eig_vecs)
print('\nEigenvalues \n%s' %eig_vals)

### 2 - Selecting Principal Components

In order to decide which eigenvector(s) can dropped without losing too much information for the construction of lower-dimensional subspace, we need to inspect the corresponding eigenvalues: The eigenvectors with the lowest eigenvalues bear the least information about the distribution of the data; those are the ones can be dropped.

In [ ]:
#sort the eigenvalues in descending order
eig_pairs = [(np.abs(eig_vals[i]),eig_vecs[:,i]) for i in range(len(eig_vals))]
for i in eig_pairs:
    print (i[0])

#### Explained Variance

After sorting the eigenpairs, the next question is “how many principal components are we going to choose for our new feature subspace?” A useful measure is the so-called “explained variance,” which can be calculated from the eigenvalues. The explained variance tells us how much information (variance) can be attributed to each of the principal components.

In [ ]:
tot = sum(eig_vals)
print(tot)
var_exp = [(i/tot)*100 for i in sorted(eig_vals,reverse=True)]
print (var_exp)

In [ ]:
#a=np.array([1,2,3,4,5,6])
#c=np.cumsum(a)
#print(c)

cum_var_exp = np.cumsum(var_exp)
print(cum_var_exp)

In [ ]:
import matplotlib.pyplot as plt
plt.bar(range(4), var_exp,  align='center',label='individual explained variance')
plt.step(range(4), cum_var_exp, where='mid',label='cumulative explained variance')
plt.ylabel('Explained variance ratio')
plt.xlabel('Principal components')
plt.legend(loc='best')
plt.tight_layout()
plt.show()

The plot above clearly shows that most of the variance (72.77% of the variance to be precise) can be explained by the first principal component alone. The second principal component still bears some information (23.03%) while the third and fourth principal components can safely be dropped without losing to much information. Together, the first two principal components contain 95.8% of the information.

In [ ]:
for i in eig_pairs:
    print (i)

In [ ]:
#Take a sequence of arrays and stack them horizontally to make a single array.
matrix_w = np.hstack((eig_pairs[0][1].reshape(4,1),
                      eig_pairs[1][1].reshape(4,1)))

print  (matrix_w)


### 3 - Projection Onto the New Feature Space

In this last step we will use the 4×2-dimensional projection matrix W to transform our samples onto the new subspace via the equation

Y=X×W, where Y is a 150×2 matrix of our transformed samples.

In [ ]:
X_std.shape,matrix_w.shape

In [ ]:
Y =X_std.dot(matrix_w)
Y

In [ ]:
Y.shape

In [ ]:
colors = ['blue', 'red', 'green']

for lab,col in zip(np.unique(y) ,colors):
    plt.scatter(Y[y==lab,0],Y[y==lab,1],label=lab,c=col)
    
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.legend(loc='lower center')
plt.tight_layout()
plt.show()
    

### Shortcut - PCA in scikit-learn

In [ ]:
pca = PCA(n_components=2)

X_train_pca = pca.fit_transform(X_std)
X_train_pca

In [ ]:
X_train_pca.shape

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
colors = ['blue', 'red', 'green']

for lab,col in zip(np.unique(y) ,colors):
    plt.scatter(X_train_pca[y==lab,0],X_train_pca[y==lab,1],label=lab,c=col)
    
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.legend(loc='lower right')
#plt.tight_layout()
plt.show()
    

In [ ]:
pca.explained_variance_ratio_